In [2]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# fix random seed for reproducibility
numpy.random.seed(7)

In [4]:
# load the dataset
dataframe = pandas.read_csv('air_passengers.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

In [5]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [6]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

96 48


In [7]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [8]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [9]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [10]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

C:\Users\neela\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
94/94 - 3s - 28ms/step - loss: 0.0561
Epoch 2/100
94/94 - 0s - 3ms/step - loss: 0.0274
Epoch 3/100
94/94 - 0s - 4ms/step - loss: 0.0208
Epoch 4/100
94/94 - 0s - 3ms/step - loss: 0.0189
Epoch 5/100
94/94 - 0s - 3ms/step - loss: 0.0175
Epoch 6/100
94/94 - 0s - 4ms/step - loss: 0.0162
Epoch 7/100
94/94 - 0s - 3ms/step - loss: 0.0152
Epoch 8/100
94/94 - 0s - 4ms/step - loss: 0.0139
Epoch 9/100
94/94 - 0s - 3ms/step - loss: 0.0128
Epoch 10/100
94/94 - 0s - 3ms/step - loss: 0.0117
Epoch 11/100
94/94 - 0s - 4ms/step - loss: 0.0108
Epoch 12/100
94/94 - 0s - 3ms/step - loss: 0.0098
Epoch 13/100
94/94 - 0s - 3ms/step - loss: 0.0087
Epoch 14/100
94/94 - 0s - 3ms/step - loss: 0.0078
Epoch 15/100
94/94 - 0s - 3ms/step - loss: 0.0069
Epoch 16/100
94/94 - 0s - 4ms/step - loss: 0.0061
Epoch 17/100
94/94 - 0s - 3ms/step - loss: 0.0054
Epoch 18/100
94/94 - 0s - 3ms/step - loss: 0.0047
Epoch 19/100
94/94 - 0s - 3ms/step - loss: 0.0041
Epoch 20/100
94/94 - 0s - 3ms/step - loss: 0.0037
Epoch 21

In [11]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [12]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [13]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))